# Making a Flower Classifier with VGG16

### Loading the VGG16 Model

In [1]:
from keras.applications import VGG16

# Setting the input size now to 64 x 64 pixel 
img_rows = 64
img_cols = 64 

# Re-loads the VGG16 model without the top or FC layers
vgg16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in vgg16.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

Using TensorFlow backend.


0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [2]:
# Function To Make our model 

def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "sigmoid")(top_model)
    return top_model

### Let's create our new model using an image size of 64 x 64

In [3]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

# Enter dataset directory

train_data_dir = 'Dataset/Train/'
validation_data_dir = 'Dataset/Validation/'

# Image generator for Training and Validation

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
                                                       
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

# Re-loads the VGG16 model without the top or FC layers
vgg16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Freeze layers
for layer in vgg16.layers:
    layer.trainable = False
    
# Number of classes in the dataset
num_classes = 2

FC_Head = addTopModel(vgg16, num_classes)

model = Model(inputs=vgg16.input, outputs=FC_Head)

print(model.summary())

Found 777 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
___________________________________________

### Training using 64 x 64 image size is MUCH faster!

In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
                   
checkpoint = ModelCheckpoint("face_recog.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.00001)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]

# Note we use a very small learning rate 
model.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(lr = 0.0001),
              metrics = ['accuracy'])

# Enter number of samples in training and validation according to dataset

nb_train_samples = 800
nb_validation_samples = 200
epochs = 20
batch_size = 32

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

model.save("face_recog.h5")

Epoch 1/20
25/25 [==============================] - 14s 564ms/step - loss: 0.3991 - accuracy: 0.8842 - val_loss: 0.1054 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.10539, saving model to face_recog.h5
Epoch 2/20
25/25 [==============================] - 14s 572ms/step - loss: 0.1950 - accuracy: 0.9787 - val_loss: 0.2301 - val_accuracy: 1.0000

Epoch 00002: val_loss did not improve from 0.10539
Epoch 3/20
25/25 [==============================] - 12s 470ms/step - loss: 0.0946 - accuracy: 0.9962 - val_loss: 0.0770 - val_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.10539 to 0.07696, saving model to face_recog.h5
Epoch 4/20
25/25 [==============================] - 12s 471ms/step - loss: 0.0578 - accuracy: 1.0000 - val_loss: 0.0060 - val_accuracy: 1.0000

Epoch 00004: val_loss improved from 0.07696 to 0.00599, saving model to face_recog.h5
Epoch 5/20
25/25 [==============================] - 11s 438ms/step - loss: 0.0352 - accuracy: 0.9975 - val_loss: 0.0017

In [5]:
# For Prediction
from keras.preprocessing import image

In [6]:
my_photo1=image.load_img("Predict.jpg", target_size=(64,64,3))

In [7]:
my_photo1=image.img_to_array(my_photo1)

In [8]:
my_photo1.shape

(64, 64, 3)

In [9]:
import numpy as np

In [10]:
my_photo1=np.expand_dims(my_photo1,axis=0)

In [11]:
my_photo1.shape

(1, 64, 64, 3)

In [12]:
from keras.applications.vgg16 import preprocess_input

In [13]:
my_photo1 = preprocess_input(my_photo1)

In [14]:
result = model.predict(my_photo1)

In [15]:
if result[0][0] == 1.0:
    print("Apeksh")
elif result[0][1] == 1.0:
    print("Divya")

Apeksh


In [16]:
model.predict(my_photo1)

array([[1.0000000e+00, 5.6321598e-37]], dtype=float32)